In [2]:
filename="input.txt"
with open(filename) as f:
    lines = [line.rstrip().split(":") for line in f.readlines()]
    for i in range(len(lines)):
        line = lines[i]
        line[0] = line[0][10:]
        x_y = line[0].replace("x=","").replace("y=","").split(", ")
        line[0] = tuple(map(int,x_y))
        line[1] = tuple(map(int,line[1][22:].replace("x=","").replace("y=","").split(", ")))

sensors = [sensor[0] for sensor in lines]
beacons = [beacon[1] for beacon in lines]
combined = sensors+beacons

def print_map(sensors,beacons):
    combined = sensors+beacons
    min_x = min(combined, key=lambda tup: tup[0])[0]
    max_x = max(combined, key=lambda tup: tup[0])[0]
    min_y = min(combined, key=lambda tup: tup[1])[1]
    max_y = max(combined, key=lambda tup: tup[1])[1]

    for y in range(min_y,max_y):
        row_string = ""
        for x in range(min_x,max_x):
            if (x,y) in sensors:
                row_string += "S"
            elif (x,y) in beacons:
                row_string += "B"
            else:
                row_string += "."
        print(row_string)


In [1]:
#print("Map: ")
#print_map(sensors,beacons)
distances = []
for i in range(len(sensors)):
    distances.append(abs(sensors[i][0]-beacons[i][0])+abs(sensors[i][1]-beacons[i][1]))
    print(sensors[i],beacons[i],distances[i])

min_x = min(combined, key=lambda tup: tup[0])[0]
max_x = max(combined, key=lambda tup: tup[0])[0]
row = 10
locations = 0 
for cur_x in range (min_x,max_x):
    #print()
    point = (cur_x,row)
    #print("New Point:", point)
    valid = False
    for i in range(len(sensors)):
        sensor = sensors[i]
        #print("Sensor: ",sensor)
        distance = abs(sensors[i][0]-point[0])+abs(sensors[i][1]-point[1])
        #print("Distances: ",distance,distances[i])
        if distance <= distances[i]:
            valid = True
            break
    if valid == True:
        locations += 1
print(locations)
    


# ranges = []
# for i in range(len(sensors)):
#     positions = []
#     sensor = sensors[i]
#     beacon = beacons[i]
#     distance = distances[i]
#     x_range = [sensor[0]-distance,sensor[0]+distance]
#     y_range = [sensor[1]-distance,sensor[1]+distance]
    

NameError: name 'sensors' is not defined